# Plotting with d3IpyPlus

## Import Libraries

In [ ]:
import pandas as pd

from functools import partial
from d3IpyPlus import *
#d3IpyPlus was found in https://github.com/maclandrol/d3IpyPlus
## One just needs to include the file d3IpyPlus.py in the same path as the Jupyter Notebook's file

## Code for Treemap

In [ ]:
#Import the DataFrame
## Here please notice that I did several changes to the DataFrame in Excel already, for convenience
df = pd.read_csv('Data_final.csv', index_col=None)

#Running the Treemamp Code
tmap = TreeMap(id=["country","sector"], size="value", color="value", legend=True, width=700)
tmap.draw(df)

In [ ]:
#To print the dump html code the below can be used. However, in this case changes were needed in the html text.
print(tmap.dump_html(df))

# Extra: Plotting with Matplotlib and Squarify
### No Interactivity in the Treemap (static graph)

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import squarify

import matplotlib
import matplotlib.pyplot as plt

In [ ]:
#Import data (different file from the above usage)
data = pd.read_csv('Data_tmap.csv', index_col=0)

#Create a Dictionary with the Data for the Different Countries
multi_df = dict()
for i in np.arange(0, len(data.columns), 3):
    test = data[data.columns[i:(i+3)]]
    test = test[pd.notnull(test[test.columns[0]])]
    test = test[1:]
    test[test.columns[0]] = pd.to_numeric(test[test.columns[0]], downcast='signed')

    #Include extra value for Other activities (with value lower than 2%)
    total_GNP = sum(test[test.columns[0]])
    test['Perc'] = test[test.columns[0]] / total_GNP
    test = test[test['Perc']>=0.013]
    test = test.sort_values(by=test.columns[0],ascending=False)
    other_value = total_GNP - sum(test[test.columns[0]])
    other_perc = str(round((1 - sum(test[test.columns[3]]))*100,2))+'%'
    test.loc['Other activities'] = [other_value, other_perc, test[test.columns[2]][0], 0]

    test = test.drop(columns=['Perc'])
    multi_df.update({test.columns[0]: test}) 

In [ ]:
# Treemap parameters
x = 0.
y = 0.
width = 100.
height = 100.
cmap = matplotlib.cm.viridis

#Define the "country" from the Dataset to which you want to create the Treemap
country = 'Portugal' 
df= multi_df[country]

#Utilise matplotlib to scale our goal numbers between the min and max, then assign this scale to our values
mini=min(df[df.columns[0]])
maxi=max(df[df.columns[0]])
norm = matplotlib.colors.Normalize(vmin=mini, vmax=maxi)
colors = [cmap(norm(value)) for value in df[df.columns[0]]]

# Labels for squares
labels = ["%s\n %s %s \n %s " % (label) for label in 
          zip(df.index, df[df.columns[0]], df[df.columns[2]], df[df.columns[1]])]

# Making the plot
fig = plt.figure(figsize=(17, 17))
fig.suptitle(df.columns[0] + "- Value Added by Sector", fontsize=20)
ax = fig.add_subplot(111, aspect="equal")
ax = squarify.plot(df[df.columns[0]], color=colors, label=labels, ax=ax, alpha=.8, linewidth=1, edgecolor='#000000')
ax.set_xticks([])
ax.set_yticks([])

plt.show()